# Cassandra
environment setup

*should be able to remove mosdef and omnia channels sooooon*

`conda create -n cassandra -y -c conda-forge -c mosdef -c omnia “python=3.7” mbuild foyer openbabel py3dmol pytest jupyterlab spglib`

`conda activate cassandra`

`git clone git@github.com:jennyfothergill/mosdef_cassandra.git`

`cd mosdef_cassandra`

`python -m pip install .`

Following example based on:

Song, M. K., & No, K. T. (2009). Grand Canonical Monte Carlo simulations of hydrogen adsorption on aluminophosphate molecular sieves. International Journal of Hydrogen Energy, 34(5), 2325–2328. https://doi.org/10.1016/j.ijhydene.2008.12.076

VPI-5 framework structure taken from https://america.iza-structure.org/IZA-SC/mat_cif.php?ID=VFI_0, oxygen atoms removed from pore by deleting atoms in cif file labeled H2O#. Then converted to conventional cell using <>. and made into a supercell.

Does Cassandra use periodic boundary conditions?

In [10]:
%%bash
# Add Cassandra to your path
export PATH=/Users/$(whoami)/Projects/Cassandra_V1.2/bin:${PATH}

In [11]:
%%bash
# Delete output of previous run
rm -rf *.in.* species* *.out* *.inp*

In [12]:
# Probably a bad idea, but also there are way too many warnings :/
import warnings

warnings.simplefilter("ignore")

In [27]:
import mosdef_cassandra as mc
import foyer
import mbuild as mb
import mosdef_cassandra.examples as ex
import numpy as np
import spglib

In [14]:
# If this fails, you need to add Cassandra to your path
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /usr/bin/python2.7
library_setup: /Users/jenny/Projects/Cassandra_V1.2/bin/library_setup.py
Cassandra: /Users/jenny/Projects/Cassandra_V1.2/bin/cassandra_gfortran.exe


In [100]:
cif_file = "VPI-5.cif"
with open(cif_file, "r") as f:
    data = [line for line in f.readlines() if not (line.isspace() or line[0] == "#")]
#print(data.index(' loop_\n'))
cell_dict = {}
for line in data:
    if line.startswith("_cell"):
        cell_dict[line.split()[0][6:]] = float(line.split()[1])
#print(cell_dict)

atom_lines = data[data.index(' loop_\n')+1:]
atom_labels = [line[11:].strip("\n") for line in atom_lines if line.startswith("_")]
atoms = []
for line in atom_lines[len(atom_labels):]:
    line_split = line.split()
    atom_dict = {}
    for label, item in zip(atom_labels,line_split):
        try:
            item = float(item)
        except ValueError:
            pass
        atom_dict[label] = item
    atoms.append(atom_dict)
#print(atom_dict)

In [112]:
pop_inds = [i for i,atom in enumerate(atoms) if atom["label"].startswith("H2O")]
for ind in pop_inds[::-1]:
    atoms.pop(ind)

In [137]:
pos = np.empty((len(atoms),3))
names = []
for i,atom in enumerate(atoms):
    names.append(atom["type_symbol"])
    pos[i,:] = [atom["fract_x"],atom["fract_y"],atom["fract_z"]]
    
#print(names, pos)

In [145]:
#print(cell_dict)
a = np.array([cell_dict["length_a"], 0, 0])

b = np.array([
    cell_dict["length_b"]*np.cos(cell_dict["angle_gamma"]),
    cell_dict["length_b"]*np.sin(cell_dict["angle_gamma"]),
    0
])

inside = (
    np.cos(cell_dict["angle_alpha"])-np.cos(cell_dict["angle_beta"])*np.cos(cell_dict["angle_gamma"])
)/np.sin(cell_dict["angle_gamma"])

c = np.array([
    cell_dict["length_c"]*np.cos(cell_dict["angle_beta"]),
    cell_dict["length_c"]*inside,
    cell_dict["length_c"]*np.sqrt(
        1-np.cos(cell_dict["angle_beta"])**2-inside**2
    ),
])

lattice = np.stack((a,b,c))
lattice

array([[18.9752    ,  0.        ,  0.        ],
       [15.44924675, 11.01721334,  0.        ],
       [-3.63136781, -1.16218437,  7.15148898]])

Next we can make a cubic unit cell like this:

![img VPI-5.png](VPI-5.png)

In [144]:
cell = (lattice, pos, numbers)
lattice, pos, numbers = spglib.standardize_cell(cell)

lattice

array([[ 8.10440000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 7.08316420e-16,  1.15676850e+01,  0.00000000e+00],
       [-3.97886481e-01, -5.78384250e+00,  1.59473040e+01]])

In [23]:
hydrogen = mb.Compound()
hydrogen.add(mb.Particle(name="H", pos=np.array([0,0,0])))
hydrogen.add(mb.Particle(name="H", pos=np.array([0,0.074,0])))
hydrogen.add_bond((hydrogen[0],hydrogen[1]))
hydrogen.visualize().show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# Load forcefields
trappe = foyer.forcefields.load_TRAPPE_UA()
oplsaa = foyer.forcefields.load_OPLSAA()

# Use foyer to apply forcefields
typed_lattice = trappe.apply(lattice)
typed_methane = oplsaa.apply(methane)

# Create box and species list
box_list = [lattice]
species_list = [typed_lattice, typed_methane]

# Since we have an occupied box we need to specify
# the number of each species present in the intial config
mols_in_boxes = [[1, 0]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moves = mc.Moves("gcmc", species_list)

custom_args = {
    "chemical_potentials": ["none", -30.0],
    "rcut_min": 0.5,
    "vdw_cutoff": 14.0,
    "charge_cutoff": 14.0,
    "coord_freq": 100,
    "prop_freq": 10,
}

mc.run(system, moves, "equilibration", 10000, 300.0, **custom_args)

In [ ]:
!head gcmc.out.xyz

In [ ]:
import numpy as np
import fresnel
import io
import PIL
import sys
import IPython

In [ ]:
file = "gcmc.out.xyz"

d_xyz = np.dtype([("atom", np.unicode_, 8), ("xyz", "d", 3)])

with open(file, "r") as f:
    lines = f.readlines()

traj = []
step = 0
while True:
    try:
        n_atoms = int(lines[step])
    except IndexError:
        break
    # print(lines[step+1]) # this should print the step
    arr = lines[step+2:step+2+n_atoms]
    frame = np.genfromtxt(arr, dtype = d_xyz)
    atoms = np.array([i["atom"] for i in frame],dtype="U8")
    xyz = np.stack([i["xyz"] for i in frame])
    traj.append((atoms,xyz))
    step += 2+n_atoms

In [ ]:
#!head gcmc.out.H
height = 29.840000000000000

In [ ]:
def render_sphere_frame(frame, height=None):

    if height is None:
        if hasattr(frame, 'configuration'):
            Ly = frame.configuration.box[1]
            height = Ly * math.sqrt(3)
        else:
            Ly = frame.box.Ly;
            height = Ly * math.sqrt(3)

    scene = fresnel.Scene(device)
    scene.lights = fresnel.light.cloudy();
    g = fresnel.geometry.Sphere(scene, position=frame[1], radius=np.ones(len(frame[0]))*0.5)
    g.material = fresnel.material.Material(solid=0.0, color=blue, primitive_color_mix=1.0, specular=1.0, roughness=0.2)
    g.outline_width = 0.07
    scene.camera = fresnel.camera.orthographic(position=(height, height, height), look_at=(0,0,0), up=(0,1,0), height=height)

    g.color[frame[0] == "C"] = blue;
    g.color[frame[0] == "_C"] = blue;
    g.color[frame[0] == "H"] = orange;

    scene.background_color = (1,1,1)

    return path_tracer.sample(scene, samples=64, light_samples=20)

In [ ]:
device = fresnel.Device(mode='cpu');
preview_tracer = fresnel.tracer.Preview(device, 300, 300)
path_tracer = fresnel.tracer.Path(device, 300, 300)

blue = fresnel.color.linear([0.25,0.5,1])*0.9;
orange = fresnel.color.linear([1.0,0.714,0.169])*0.9

def display_movie(frame_gen, traj, gif = None):
    a = frame_gen(traj[0], height = 30);

    if tuple(map(int, (PIL.__version__.split(".")))) < (3,4,0):
        print("Warning! Movie display output requires pillow 3.4.0 or newer.")
        print("Older versions of pillow may only display the first frame.")

    im0 = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB').convert("P", palette=PIL.Image.ADAPTIVE);
    ims = [];
    for f in traj[1:]:
        a = frame_gen(f, height = 30);
        im = PIL.Image.fromarray(a[:,:, 0:3], mode='RGB')
        im_p = im.quantize(palette=im0);
        ims.append(im_p)

    if gif:
        im0.save(gif, 'gif', save_all=True, append_images=ims, duration=1000, loop=0)
        return
    if (sys.version_info[0] >= 3):
        size = len(io.BytesIO().getbuffer())/1024;
        if (size > 2000):
            print("Size:", size, "KiB")

    return IPython.display.display(IPython.display.Image(data=io.BytesIO().getvalue()))

In [ ]:
#render_sphere_frame(traj[2], height)

display_movie(render_sphere_frame, traj, "traj.gif")

In [ ]:
# gcmc.out.H - box
# gcmc.out.xyz - particle coordinates
# species#.mcf - connectivity
# Cassandra treats no bonds as fixed - check the inp file (max distances for the frozen species should be 0)
# this file species2/frag1/frag1.dat contains the fragment configurations 
# --changing the temp (300) to natoms (5 for methane) makes it viewable by vmd (watch the fragment wiggle around)
# chk - restart (mc.restart)
# log - energies, etc
# properties - thermo properties
# cassandra uses fixed bonds -- make sure ring systems have correct bonds!
!ls